In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

!pip install keras_tuner
import keras_tuner as kt

from sklearn import linear_model
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [2]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/MATSCI176/Final\ Project'
%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MATSCI176/Final Project


In [3]:
# Data preprocessing for Nasa battery datasets
# %run convert_nasa_dataset_discharge.py

In [4]:
# Data Pre-processing

# Load dataset
file_name = "nasa_batteries_processed/data.csv"
absolute_path = os.path.abspath(file_name)  # Combine folder name and file name to create the full file path
df = pd.read_csv(absolute_path)
#df["load_current"] = "N/A"

#battery_id = [5, 6, 7, 18, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 53, 54, 55, 56]
battery_id = [5, 6, 7, 18, 25, 29, 31, 34, 36, 45, 46, 47, 48, 54, 55, 56]

value_map = {5: 2, 6: 2, 7: 2, 18: 2, 25: 4, 29: 4, 31: 1.5, 34: 4, 36: 2, 45: 1, 46: 1, 47: 1, 48: 1, 54: 2, 55: 2, 56: 2}
df["load_current"] = df["battery_id"].map(value_map)

# Drop rows where 'SOH' has a zero value
df = df[df['soh'] != 0]

# Drop rows where 'column_name' is equal to a specific value
#df = df[df['cycle_id'] != 0]

# Assuming your dataframe is sorted by 'cycle_id' or any relevant column
df['prev_soh'] = df['soh'].shift(1)  # Get the previous SOH value
df['next_soh'] = df['soh'].shift(-1)  # Get the next SOH value

# Calculate percentage differences
df['prev_diff'] = abs(df['soh'] - df['prev_soh'])
df['next_diff'] = abs(df['soh'] - df['next_soh'])

# Add a check for the first row (compare the first row with the second)
df['first_row_diff'] = abs(df['soh'] - df['next_soh'])
df.loc[0, 'prev_diff'] = df.loc[0, 'first_row_diff']  # For the first row, set the previous difference to compare with the second

# Filter out rows where the SOH difference exceeds 10% in the previous or next row
df_filtered = df[(df['prev_diff'] <= 10) & (df['next_diff'] <= 10)]

# Drop the temporary columns used for calculation
df_filtered.drop(columns=['prev_soh', 'next_soh', 'prev_diff', 'next_diff', 'first_row_diff'], inplace=True)

# Training cycles
n_cycles = 40

<ipython-input-4-0498139c99f3>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['prev_soh', 'next_soh', 'prev_diff', 'next_diff', 'first_row_diff'], inplace=True)


In [5]:
# Data Pre-processing -> Normalization

# Normalize SOH (optional)
scaler = MinMaxScaler()
df_filtered['SOH_scaled'] = scaler.fit_transform(df_filtered[['soh']])

# Train-Test Split Function
def prepare_data(df, battery_id):
    battery_data = df[df['battery_id'] == battery_id].sort_values(by="cycle_id")
    train = battery_data[battery_data["cycle_id"] <= n_cycles]
    test = battery_data[battery_data["cycle_id"] > n_cycles]
    return train, test

# Apply to all batteries
train_list, test_list = [], []
for b in battery_id:
    train, test = prepare_data(df_filtered, b)
    train_list.append(train)
    test_list.append(test)

train_df = pd.concat(train_list)
test_df = pd.concat(test_list)

<ipython-input-5-1531a34d3b35>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['SOH_scaled'] = scaler.fit_transform(df_filtered[['soh']])


In [6]:
# Feature Selection -> single feature
features = ["cycle_id"]  # You can add more time-based features
X_train, y_train = train_df[features], train_df["SOH_scaled"]
X_test, y_test = test_df[features], test_df["SOH_scaled"]

In [7]:
# XGBoost Model -> calculate MAE for entire dataset
# Use only cycle_id as feature for training

# Train XGBoost Model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict
y_pred = xgb_model.predict(X_test)

# Evaluate Performance
mae = mean_absolute_error(y_test, y_pred)
print(f"XGBoost MAE (single feature): {mae}")

XGBoost MAE (single feature): 0.10692547382669404


In [8]:
# Hyperparameter tuning for XGBoost model using single feature

# Define parameter grid
param_grid = {
    "n_estimators": [100, 200, 500],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 5, 7],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}

# Create model
xgb_model = XGBRegressor(random_state=42)

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=xgb_model, param_grid=param_grid,
    cv=5, scoring='neg_mean_absolute_error',
    n_jobs=-1, verbose=1
)

grid_search.fit(X_train, y_train)

# Best parameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Optimized XGBoost MAE (single feature): {mae}")

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best Hyperparameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
Optimized XGBoost MAE (single feature): 0.10624439054066982


In [9]:
# Random Forest Model -> calculate MAE for entire dataset
# Use only cycle_id as feature for training

# Train Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Evaluate
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Random Forest MAE (single feature): {mae_rf}")

Random Forest MAE (single feature): 0.1070223724333034


In [10]:
# Hyperparameter tuning for Random Forest model using single feature

# Define parameter grid
param_grid = {
    "n_estimators": [50, 100, 200],  # Number of trees in the forest
    "max_depth": [None, 10, 20],  # Maximum depth of the trees
    "min_samples_split": [2, 5, 10],  # Minimum samples required to split a node
    "min_samples_leaf": [1, 2, 4],  # Minimum samples required in a leaf node
    "max_features": ["sqrt", "log2"]  # Number of features to consider at each split
}

# Initialize Random Forest Model
rf_model = RandomForestRegressor(random_state=42)

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=rf_model, param_grid=param_grid,
    cv=5, scoring='neg_mean_absolute_error',
    n_jobs=-1, verbose=1
)

# Fit the model
grid_search.fit(X_train, y_train)

# Get best parameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the best model on the test set
best_rf_model = grid_search.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)

# Calculate MAE
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Optimized Random Forest MAE (single feature): {mae_rf}")

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best Hyperparameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Optimized Random Forest MAE (single feature): 0.10678655404209662


In [11]:
# Feature Selection -> multiple features
features = ["ambient_temperature", "cutoff_voltage", "cycle_id", "load_current"]  # You can add more time-based features
X_train, y_train = train_df[features], train_df["SOH_scaled"]
X_test, y_test = test_df[features], test_df["SOH_scaled"]

In [12]:
# XGBoost Model -> calculate MAE for entire dataset
# Use multiple features for training

# Train XGBoost Model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)


# Predict
y_pred = xgb_model.predict(X_test)

# Evaluate Performance
mae = mean_absolute_error(y_test, y_pred)
print(f"XGBoost MAE (multiple feature): {mae}")

XGBoost MAE (multiple feature): 0.07291767470569481


In [13]:
# Hyperparameter tuning for XGBoost model using multiple features

# Define parameter grid
param_grid = {
    "n_estimators": [100, 200, 500],  # Number of boosting rounds
    "learning_rate": [0.01, 0.1, 0.2],  # Step size shrinkage
    "max_depth": [3, 5, 7],  # Maximum depth of a tree
    "subsample": [0.7, 0.8, 1.0],  # Fraction of samples used per boosting round
    "colsample_bytree": [0.7, 0.8, 1.0],  # Fraction of features used per tree
    "gamma": [0, 0.1, 0.2],  # Minimum loss reduction to make a split
    "reg_alpha": [0, 0.1, 1],  # L1 regularization term
    "reg_lambda": [1, 2, 5]  # L2 regularization term
}

# Initialize XGBoost Model
xgb_model = XGBRegressor(random_state=42)

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=xgb_model, param_grid=param_grid,
    cv=5, scoring='neg_mean_absolute_error',
    n_jobs=-1, verbose=1
)

# Fit the model
grid_search.fit(X_train, y_train)

# Get best parameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the best model on the test set
best_xgb_model = grid_search.best_estimator_
y_pred_xgb = best_xgb_model.predict(X_test)

# Calculate MAE
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"Optimized XGBoost MAE (multiple features): {mae_xgb}")

Fitting 5 folds for each of 6561 candidates, totalling 32805 fits
Best Hyperparameters: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 5, 'subsample': 0.8}
Optimized XGBoost MAE (multiple features): 0.07286844942668136


In [14]:
# Random Forest Model -> calculate MAE for entire dataset
# Use multiple features for training

# Train Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Evaluate
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Random Forest MAE (multiple features): {mae_rf}")

Random Forest MAE (multiple features): 0.07452653562603848


In [15]:
# Hyperparameter tuning for Random Forest model using multiple features

# Define parameter grid
param_grid = {
    "n_estimators": [100, 200, 500],  # Number of trees in the forest
    "max_depth": [None, 10, 20],  # Maximum depth of the trees
    "min_samples_split": [2, 5, 10],  # Minimum samples required to split a node
    "min_samples_leaf": [1, 2, 4],  # Minimum samples required in a leaf node
    "max_features": ["sqrt", "log2"],  # Number of features to consider at each split
    "bootstrap": [True, False]  # Whether to use bootstrapping samples
}

# Initialize Random Forest Model
rf_model = RandomForestRegressor(random_state=42)

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=rf_model, param_grid=param_grid,
    cv=5, scoring='neg_mean_absolute_error',
    n_jobs=-1, verbose=1
)

# Fit the model
grid_search.fit(X_train, y_train)

# Get best parameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the best model on the test set
best_rf_model = grid_search.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)

# Calculate MAE
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Optimized Random Forest MAE: {mae_rf}")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Hyperparameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 500}
Optimized Random Forest MAE: 0.07562066476114607


In [16]:
# LSTM Model -> calculate MAE for entire dataset
# Use only battery_id as feature for training

# Reshape Data for LSTM
def reshape_for_lstm(df_filtered):
    X, y = [], []
    grouped = df_filtered.groupby("battery_id")
    for _, group in grouped:
        cycles = group["cycle_id"].values.reshape(-1, 1)  # Use cycle_id as time-series input
        soh = group["SOH_scaled"].values
        for i in range(len(cycles) - n_cycles):  # Use first 30 cycles as input
            X.append(cycles[i:i+n_cycles])
            y.append(soh[i+n_cycles])
    return np.array(X), np.array(y)

X_train_lstm, y_train_lstm = reshape_for_lstm(train_df)
X_test_lstm, y_test_lstm = reshape_for_lstm(test_df)

# Reshape to (samples, timesteps, features)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], 1)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], 1)

# Build LSTM Model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(n_cycles, 1)),
    LSTM(50, return_sequences=False),
    Dense(25, activation="relu"),
    Dense(1)  # Output layer
])

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

# Train Model
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Predict
y_pred_lstm = lstm_model.predict(X_test_lstm)

# Evaluate
mae_lstm = mean_absolute_error(y_test_lstm, y_pred_lstm)
print(f"LSTM MAE (single feature): {mae_lstm}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 0.6360 - val_loss: 0.2112
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.2851 - val_loss: 0.0898
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.0921 - val_loss: 0.0318
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 714ms/step - loss: 0.0049 - val_loss: 0.0294
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - loss: 0.0186 - val_loss: 0.0549
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 712ms/step - loss: 0.0692 - val_loss: 0.0746
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - loss: 0.0931 - val_loss: 0.0757
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 716ms/step - loss: 0.0811 - val_loss: 0.0629
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 980ms/step - loss: 0.0504 - val_loss: 0.0453
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 895ms/step - loss: 0.0205 - val_loss: 0.0301
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 988ms/step - loss: 0.0030 - val_loss: 0.0199
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - loss: 7.6719e-04 - val_loss:

In [17]:
# Hyperparameter tuning for LSTM model using single feature

# Set random seed for reproducibility
seed = 99
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

# Ensure TensorFlow deterministic behavior (use if running on GPU)
tf.config.experimental.enable_op_determinism()

# Hyperparameters
batch_size = 16
epochs = 50

# Standardize cycle_id for stability
scaler = StandardScaler()

# Function to reshape data for LSTM
def reshape_for_lstm(df_filtered, n_cycles=40):
    X, y = [], []
    grouped = df_filtered.groupby("battery_id")
    for _, group in grouped:
        cycles = group["cycle_id"].values.reshape(-1, 1)  # Use cycle_id as time-series input
        soh = group["SOH_scaled"].values
        for i in range(len(cycles) - n_cycles):  # Use first 30 cycles as input
            X.append(cycles[i:i + n_cycles])
            y.append(soh[i + n_cycles])
    return np.array(X), np.array(y)

# Prepare data
X_train_lstm, y_train_lstm = reshape_for_lstm(train_df)
X_test_lstm, y_test_lstm = reshape_for_lstm(test_df)

# Reshape for LSTM (samples, timesteps, features)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], 1)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], 1)

# Define model builder function
def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(
        units=hp.Int('units_1', min_value=32, max_value=128, step=32),
        return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)
    ))
    model.add(LSTM(
        units=hp.Int('units_2', min_value=32, max_value=128, step=32),
        return_sequences=False
    ))
    model.add(Dense(hp.Int('dense_units', min_value=16, max_value=64, step=16), activation="relu"))
    model.add(Dense(1))  # Output layer

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])),
        loss="mse"
    )
    return model

# Initialize tuner
tuner = kt.BayesianOptimization(
    build_lstm_model,
    objective="val_loss",
    max_trials=10,  # Number of different hyperparameter combinations to try
    directory="lstm_tuning",
    project_name="battery_lstm"
)

# Run hyperparameter search
tuner.search(X_train_lstm, y_train_lstm, epochs=30, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

# Train best model
best_lstm_model = tuner.hypermodel.build(best_hps)
best_lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Predict
y_pred_lstm = best_lstm_model.predict(X_test_lstm)

# Evaluate
mae_lstm = mean_absolute_error(y_test_lstm, y_pred_lstm)
print(f"Optimized LSTM MAE (single feature): {mae_lstm}")

# Get the best hyperparameters from the tuning process
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameter values
print("Best Hyperparameters:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

Reloading Tuner from lstm_tuning/battery_lstm/tuner0.json
Best Hyperparameters: {'units_1': 128, 'units_2': 64, 'dense_units': 32, 'learning_rate': 0.0001}
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.3149 - val_loss: 0.1952
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 769ms/step - loss: 0.2282 - val_loss: 0.1527
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.1553 - val_loss: 0.1163
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.0967 - val_loss: 0.0860
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.0525 - val_loss: 0.0617
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.0226 - val_loss: 0.0427
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.0057 - val_loss: 0.0290
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 5.5406e-05 - val_loss: 0.0202
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 632ms/step - loss: 0.0028 - val_loss: 0.0154
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 735ms/step - loss: 0.0106 - val_loss: 0.0135
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 761ms/step - loss: 0.0198 - val_loss: 0.0130
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 742ms/step - loss: 0.0272 - val_loss: 0.0130
Epoch 13/50
1/1 ━

In [18]:
# LSTM Model -> calculate MAE for entire dataset
# Use multiple features for training

# Features to use
features = ["ambient_temperature", "cutoff_voltage", "cycle_id", "load_current"]

# Reshape Data for LSTM
def reshape_for_lstm(df_filtered):
    X, y = [], []
    grouped = df_filtered.groupby("battery_id")
    for _, group in grouped:
        feature_values = group[features].values  # Extract multiple features
        soh = group["SOH_scaled"].values
        for i in range(len(feature_values) - n_cycles):  # Use first 30 cycles as input
            X.append(feature_values[i:i+n_cycles])
            y.append(soh[i+n_cycles])  # Predict SOH at cycle i+30
    return np.array(X), np.array(y)

# Prepare LSTM Training & Testing Data
X_train_lstm, y_train_lstm = reshape_for_lstm(train_df)
X_test_lstm, y_test_lstm = reshape_for_lstm(test_df)

# Reshape to (samples, timesteps, features)
num_features = len(features)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], num_features)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], num_features)

# Build LSTM Model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(n_cycles, num_features)),  # Adjust input shape
    LSTM(50, return_sequences=False),
    Dense(25, activation="relu"),
    Dense(1)  # Output layer
])

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

# Train Model
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Predict
y_pred_lstm = lstm_model.predict(X_test_lstm)

# Evaluate
mae_lstm = mean_absolute_error(y_test_lstm, y_pred_lstm)
print(f"LSTM MAE (multiple features): {mae_lstm:.4f}")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.5834 - val_loss: 0.4568
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 740ms/step - loss: 0.4619 - val_loss: 0.3533
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 713ms/step - loss: 0.3486 - val_loss: 0.2652
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step - loss: 0.2364 - val_loss: 0.1831
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 895ms/step - loss: 0.1397 - val_loss: 0.1169
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.0669 - val_loss: 0.0684
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 994ms/step - loss: 0.0203 - val_loss: 0.0361
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 631ms/step - loss: 0.0010 - val_loss: 0.0192
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 718ms/step - loss: 0.0052 - val_loss: 0.0156
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.0232 - val_loss: 0.0185
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.0375 - val_loss: 0.0211
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.0393 - val_loss: 0.0216
Epoch 13/50
1/1

In [19]:
# Hyperparameter tuning for LSTM model using multiple features

# Set random seed for reproducibility
seed = 99
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

# Ensure TensorFlow deterministic behavior (use if running on GPU)
tf.config.experimental.enable_op_determinism()

# Hyperparameters
batch_size = 16
epochs = 50

# Standardize cycle_id for stability
scaler = StandardScaler()

# Features to use
features = ["ambient_temperature", "cutoff_voltage", "cycle_id", "load_current"]

# Function to reshape data for LSTM
def reshape_for_lstm(df_filtered, n_cycles=30):
    X, y = [], []
    grouped = df_filtered.groupby("battery_id")
    for _, group in grouped:
        feature_values = group[features].values  # Extract multiple features
        soh = group["SOH_scaled"].values
        for i in range(len(feature_values) - n_cycles):  # Use first 30 cycles as input
            X.append(feature_values[i:i + n_cycles])
            y.append(soh[i + n_cycles])  # Predict SOH at cycle i+30
    return np.array(X), np.array(y)

# Prepare LSTM Training & Testing Data
X_train_lstm, y_train_lstm = reshape_for_lstm(train_df)
X_test_lstm, y_test_lstm = reshape_for_lstm(test_df)

# Reshape to (samples, timesteps, features)
num_features = len(features)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], num_features)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], num_features)

# Define model builder function for tuning
def build_lstm_model(hp):
    model = Sequential()

    # First LSTM layer
    model.add(LSTM(
        units=hp.Int('units_1', min_value=32, max_value=128, step=32),
        return_sequences=True,
        input_shape=(X_train_lstm.shape[1], num_features)
    ))

    # Second LSTM layer
    model.add(LSTM(
        units=hp.Int('units_2', min_value=32, max_value=128, step=32),
        return_sequences=False
    ))

    # Dropout layer for regularization
    model.add(Dropout(hp.Choice('dropout', [0.0, 0.2, 0.4])))

    # Dense layer
    model.add(Dense(hp.Int('dense_units', min_value=16, max_value=64, step=16), activation="relu"))

    # Output layer
    model.add(Dense(1))

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])),
        loss="mse"
    )

    return model

# Initialize tuner
tuner = kt.BayesianOptimization(
    build_lstm_model,
    objective="val_loss",
    max_trials=10,  # Number of different hyperparameter combinations to try
    directory="lstm_tuning",
    project_name="battery_lstm_multifeature"
)

# Run hyperparameter search
tuner.search(X_train_lstm, y_train_lstm, epochs=30, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

# Train the best model
best_lstm_model = tuner.hypermodel.build(best_hps)
best_lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Predict
y_pred_lstm = best_lstm_model.predict(X_test_lstm)

# Evaluate
mae_lstm = mean_absolute_error(y_test_lstm, y_pred_lstm)
print(f"Optimized LSTM MAE (multiple features): {mae_lstm:.4f}")

# Print the best hyperparameter values
print("Best Hyperparameters:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

Reloading Tuner from lstm_tuning/battery_lstm_multifeature/tuner0.json
Best Hyperparameters: {'units_1': 96, 'units_2': 64, 'dropout': 0.2, 'dense_units': 48, 'learning_rate': 0.0005}
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 202ms/step - loss: 0.2280 - val_loss: 0.0101
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - loss: 0.0394 - val_loss: 0.0838
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - loss: 0.0229 - val_loss: 0.0307
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0223 - val_loss: 0.0323
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - loss: 0.0196 - val_loss: 0.0439
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0126 - val_loss: 0.0348
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - loss: 0.0152 - val_loss: 0.0371
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - loss: 0.0174 - val_loss: 0.0389
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - loss: 0.0160 - val_loss: 0.0360
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - loss: 0.0133 - val_loss: 0.0315
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - loss: 0.0143 - val_loss: 0.0315
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - loss: 0.0109 - val_loss: 0.0324
Ep

In [20]:
# Bi-directional LSTM -> calculate MAE for entire dataset
# Use only battery_id as feature for training

from tensorflow.keras.layers import Bidirectional

# Reshape Data for LSTM
def reshape_for_lstm(df_filtered, n_cycles):
    X, y = [], []
    grouped = df_filtered.groupby("battery_id")
    for _, group in grouped:
        cycles = group["cycle_id"].values.reshape(-1, 1)  # Use cycle_id as time-series input
        soh = group["SOH_scaled"].values
        for i in range(len(cycles) - n_cycles):  # Use first 30 cycles as input
            X.append(cycles[i:i+n_cycles])
            y.append(soh[i+n_cycles])
    return np.array(X), np.array(y)

X_train_lstm, y_train_lstm = reshape_for_lstm(train_df, 30)
X_test_lstm, y_test_lstm = reshape_for_lstm(test_df, 30)

# Reshape to (samples, timesteps, features)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], 1)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], 1)

# Build Bi-LSTM Model
bi_lstm_model = Sequential([
    Bidirectional(LSTM(50, return_sequences=True), input_shape=(30, 1)),  # Bidirectional LSTM layer
    Bidirectional(LSTM(50, return_sequences=False)),  # Another Bidirectional LSTM layer
    Dense(25, activation="relu"),
    Dense(1)  # Output layer
])

bi_lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

# Train Model
bi_lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Predict
y_pred_bi_lstm = bi_lstm_model.predict(X_test_lstm)

# Evaluate
mae_bi_lstm = mean_absolute_error(y_test_lstm, y_pred_bi_lstm)
print(f"Bi-LSTM MAE (single feature): {mae_bi_lstm}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 201ms/step - loss: 0.4710 - val_loss: 0.0374
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - loss: 0.0615 - val_loss: 0.0960
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 240ms/step - loss: 0.0425 - val_loss: 0.0472
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - loss: 0.0368 - val_loss: 0.0683
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - loss: 0.0364 - val_loss: 0.0532
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 0.0359 - val_loss: 0.0606
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - loss: 0.0358 - val_loss: 0.0554
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - loss: 0.0358 - val_loss: 0.0572
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 0.0357 - val_loss: 0.0554
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - loss: 0.0358 - val_loss: 0.0548
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 0.0357 - val_loss: 0.0540
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - loss: 0.0357 - val_loss: 0.0527


In [21]:
# Hyperparameter tuning for Bi-LSTM model using single feature

# Function to reshape data for LSTM
def reshape_for_lstm(df_filtered, n_cycles=30):
    X, y = [], []
    grouped = df_filtered.groupby("battery_id")
    for _, group in grouped:
        cycles = group["cycle_id"].values.reshape(-1, 1)  # Use cycle_id as time-series input
        soh = group["SOH_scaled"].values
        for i in range(len(cycles) - n_cycles):  # Use first 30 cycles as input
            X.append(cycles[i:i + n_cycles])
            y.append(soh[i + n_cycles])
    return np.array(X), np.array(y)

# Prepare LSTM Training & Testing Data
X_train_lstm, y_train_lstm = reshape_for_lstm(train_df)
X_test_lstm, y_test_lstm = reshape_for_lstm(test_df)

# Reshape to (samples, timesteps, features)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], 1)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], 1)

# Define model builder function for tuning
def build_bi_lstm_model(hp):
    model = Sequential()

    # First Bi-LSTM layer
    model.add(Bidirectional(LSTM(
        units=hp.Int('units_1', min_value=32, max_value=128, step=32),
        return_sequences=True
    ), input_shape=(X_train_lstm.shape[1], 1)))

    # Second Bi-LSTM layer
    model.add(Bidirectional(LSTM(
        units=hp.Int('units_2', min_value=32, max_value=128, step=32),
        return_sequences=False
    )))

    # Dropout layer for regularization
    model.add(Dropout(hp.Choice('dropout', [0.0, 0.2, 0.4])))

    # Dense layer
    model.add(Dense(hp.Int('dense_units', min_value=16, max_value=64, step=16), activation="relu"))

    # Output layer
    model.add(Dense(1))

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])),
        loss="mse"
    )

    return model

# Initialize tuner
tuner = kt.BayesianOptimization(
    build_bi_lstm_model,
    objective="val_loss",
    max_trials=10,  # Number of different hyperparameter combinations to try
    directory="bi_lstm_tuning",
    project_name="battery_bi_lstm"
)

# Run hyperparameter search
tuner.search(X_train_lstm, y_train_lstm, epochs=30, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

# Train the best model
best_bi_lstm_model = tuner.hypermodel.build(best_hps)
best_bi_lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Predict
y_pred_bi_lstm = best_bi_lstm_model.predict(X_test_lstm)

# Evaluate
mae_bi_lstm = mean_absolute_error(y_test_lstm, y_pred_bi_lstm)
print(f"Optimized Bi-LSTM MAE (single feature): {mae_bi_lstm:.4f}")

Reloading Tuner from bi_lstm_tuning/battery_bi_lstm/tuner0.json
Best Hyperparameters: {'units_1': 64, 'units_2': 128, 'dropout': 0.2, 'dense_units': 64, 'learning_rate': 0.001}
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 423ms/step - loss: 0.4513 - val_loss: 0.0375
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 235ms/step - loss: 0.0484 - val_loss: 0.0482
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 237ms/step - loss: 0.0529 - val_loss: 0.0137
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 238ms/step - loss: 0.0420 - val_loss: 0.0234
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 237ms/step - loss: 0.0380 - val_loss: 0.0261
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 270ms/step - loss: 0.0463 - val_loss: 0.0176
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step - loss: 0.0411 - val_loss: 0.0358
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 240ms/step - loss: 0.0368 - val_loss: 0.0301
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 232ms/step - loss: 0.0462 - val_loss: 0.0181
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - loss: 0.0408 - val_loss: 0.0230
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 163ms/step - loss: 0.0383 - val_loss: 0.0208
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 235ms/step - loss: 0.0459 - val_loss: 0.0199

In [22]:
# Bi-directional LSTM -> calculate MAE for entire dataset
# Use multiple features for training

# Features to use
features = ["ambient_temperature", "cutoff_voltage", "cycle_id", "load_current"]

# Reshape Data for Bi-LSTM
def reshape_for_lstm(df_filtered, n_cycles):
    X, y = [], []
    grouped = df_filtered.groupby("battery_id")
    for _, group in grouped:
        feature_values = group[features].values  # Extract multiple features
        soh = group["SOH_scaled"].values
        for i in range(len(feature_values) - n_cycles):  # Use first 30 cycles as input
            X.append(feature_values[i:i+n_cycles])
            y.append(soh[i+n_cycles])  # Predict SOH at cycle i+30
    return np.array(X), np.array(y)

# Prepare Bi-LSTM Training & Testing Data
n_cycles = 30  # Define the number of cycles used for training
X_train_lstm, y_train_lstm = reshape_for_lstm(train_df, n_cycles)
X_test_lstm, y_test_lstm = reshape_for_lstm(test_df, n_cycles)

# Reshape to (samples, timesteps, features)
num_features = len(features)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], num_features)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], num_features)

# Build Bi-LSTM Model
bi_lstm_model = Sequential([
    Bidirectional(LSTM(50, return_sequences=True), input_shape=(n_cycles, num_features)),  # Bidirectional LSTM layer
    Bidirectional(LSTM(50, return_sequences=False)),  # Another Bidirectional LSTM layer
    Dense(25, activation="relu"),
    Dense(1)  # Output layer
])

bi_lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

# Train Model
bi_lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Predict
y_pred_bi_lstm = bi_lstm_model.predict(X_test_lstm)

# Evaluate
mae_bi_lstm = mean_absolute_error(y_test_lstm, y_pred_bi_lstm)
print(f"Bi-LSTM MAE (multiple features): {mae_bi_lstm:.4f}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 222ms/step - loss: 0.3779 - val_loss: 0.0198
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 247ms/step - loss: 0.0292 - val_loss: 0.0771
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 0.0117 - val_loss: 0.0167
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - loss: 0.0104 - val_loss: 0.0387
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - loss: 0.0066 - val_loss: 0.0201
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 0.0083 - val_loss: 0.0378
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - loss: 0.0059 - val_loss: 0.0274
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - loss: 0.0069 - val_loss: 0.0371
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 0.0055 - val_loss: 0.0333
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - loss: 0.0062 - val_loss: 0.0387
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 205ms/step - loss: 0.0056 - val_loss: 0.0374
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - loss: 0.0056 - val_loss: 0.0400


In [23]:
# Hyperparameter tuning for Bi-LSTM model using multiple features

# Features to use
features = ["ambient_temperature", "cutoff_voltage", "cycle_id", "load_current"]

# Reshape Data for Bi-LSTM
def reshape_for_lstm(df_filtered, n_cycles=30):
    X, y = [], []
    grouped = df_filtered.groupby("battery_id")
    for _, group in grouped:
        feature_values = group[features].values  # Extract multiple features
        soh = group["SOH_scaled"].values
        for i in range(len(feature_values) - n_cycles):  # Use first 30 cycles as input
            X.append(feature_values[i:i + n_cycles])
            y.append(soh[i + n_cycles])  # Predict SOH at cycle i+30
    return np.array(X), np.array(y)

# Prepare Bi-LSTM Training & Testing Data
n_cycles = 30  # Define the number of cycles used for training
X_train_lstm, y_train_lstm = reshape_for_lstm(train_df, n_cycles)
X_test_lstm, y_test_lstm = reshape_for_lstm(test_df, n_cycles)

# Reshape to (samples, timesteps, features)
num_features = len(features)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], num_features)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], num_features)

# Define model builder function for tuning
def build_bi_lstm_model(hp):
    model = Sequential()

    # First Bi-LSTM layer
    model.add(Bidirectional(LSTM(
        units=hp.Int('units_1', min_value=32, max_value=128, step=32),
        return_sequences=True
    ), input_shape=(n_cycles, num_features)))

    # Second Bi-LSTM layer
    model.add(Bidirectional(LSTM(
        units=hp.Int('units_2', min_value=32, max_value=128, step=32),
        return_sequences=False
    )))

    # Dropout layer for regularization
    model.add(Dropout(hp.Choice('dropout', [0.0, 0.2, 0.4])))

    # Dense layer
    model.add(Dense(hp.Int('dense_units', min_value=16, max_value=64, step=16), activation="relu"))

    # Output layer
    model.add(Dense(1))

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])),
        loss="mse"
    )

    return model

# Initialize tuner
tuner = kt.BayesianOptimization(
    build_bi_lstm_model,
    objective="val_loss",
    max_trials=10,  # Number of different hyperparameter combinations to try
    directory="bi_lstm_tuning",
    project_name="battery_bi_lstm_multiple_features"
)

# Run hyperparameter search
tuner.search(X_train_lstm, y_train_lstm, epochs=30, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

# Train the best model
best_bi_lstm_model = tuner.hypermodel.build(best_hps)
best_bi_lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=16, validation_data=(X_test_lstm, y_test_lstm))

# Predict
y_pred_bi_lstm = best_bi_lstm_model.predict(X_test_lstm)

# Evaluate
mae_bi_lstm = mean_absolute_error(y_test_lstm, y_pred_bi_lstm)
print(f"Optimized Bi-LSTM MAE (multiple features): {mae_bi_lstm:.4f}")

Reloading Tuner from bi_lstm_tuning/battery_bi_lstm_multiple_features/tuner0.json
Best Hyperparameters: {'units_1': 96, 'units_2': 32, 'dropout': 0.4, 'dense_units': 16, 'learning_rate': 0.001}
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 443ms/step - loss: 0.5573 - val_loss: 0.0087
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - loss: 0.0769 - val_loss: 0.0314
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - loss: 0.0499 - val_loss: 0.0112
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - loss: 0.0394 - val_loss: 0.0230
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - loss: 0.0305 - val_loss: 0.0274
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - loss: 0.0344 - val_loss: 0.0252
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - loss: 0.0320 - val_loss: 0.0351
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - loss: 0.0296 - val_loss: 0.0277
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - loss: 0.0216 - val_loss: 0.0236
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 225ms/step - loss: 0.0272 - val_loss: 0.0397
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 259ms/step - loss: 0.0299 - val_loss: 0.0509
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - loss: 0.0265 - val_loss: 0.0667